## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Tables

### Links: 

[[t_subscribtions_citizens_by_companies_and_cities_address_ch]]

[[uk_addresses_st_partner]]

[[uk_st_partner]]

[[uk_dir_partner]]

[[entries_installation_points_dir_partner]]
___

In [32]:
query_text = """--sql
CREATE TABLE db1.t_uk_adresses_subscribtions
(
    `report_date` Date,
    `partner_uuid_uk` String,
    `name` String,
    `address_uuid` String,
    `partner_uuid` String,
    `partner_uk_email` String,
    `company_name` String,
    `partner_lk` UInt32,
    `activated_citizen_id` UInt32,
    `citizen_id_in_flat_with_subscriptions` UInt32,
    `subscribed_citizen_id` UInt32,
    `payments_amount` UInt32,
    `city` String,
    `full_address` String,
    `flats_count` UInt16,
    `entrance_number` String,
    `building_number` String,
    `streat_name` String 
)
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [33]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_uk_adresses_subscribtions_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_uk_adresses_subscribtions AS
WITH d_range AS (
SELECT
	report_date
FROM db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
),
uk_st AS(
SELECT
	`report_date`,
     uk_s_p.`partner_uuid` AS partner_uuid_uk,
    `business_partner_uuid` AS partner_uuid,
    `updated_at`,
    `partner_uk_email`,
    `name`
FROM db1.uk_st_partner_ch AS uk_s_p
LEFT JOIN db1.uk_dir_partner_ch AS uk_d_p ON uk_d_p.partner_uuid_uk = uk_s_p.partner_uuid
WHERE report_date IN d_range
)
--
SELECT
    uk_addr.`report_date` AS report_date,
    uk_addr.`partner_uuid_uk` AS partner_uuid_uk,
    `name`,
    uk_addr.`address_uuid` AS address_uuid,
    uk_st.partner_uuid AS partner_uuid,
    partner_uk_email,
    cdp.company_name AS company_name,
    cdp.partner_lk AS partner_lk,
    activated_citizen_id,
    citizen_id_in_flat_with_subscriptions,
    subscribed_citizen_id,
    payments_amount,
    t_sub.city AS city,
    t_sub.full_address AS full_address,
    flats_count,
    `entrance_number`,
    `building_number`,
    `streat_name`    
FROM db1.uk_addresses_st_partner_ch AS uk_addr
LEFT JOIN uk_st ON uk_st.report_date = uk_addr.report_date 
				AND uk_st.partner_uuid_uk = uk_addr.partner_uuid_uk 
LEFT JOIN db1.companies_dir_partner_ch AS cdp ON cdp.partner_uuid = uk_st.partner_uuid
LEFT JOIN db1.t_subscribtions_citizens_by_companies_and_cities_address_ch AS t_sub
	ON t_sub.address_uuid = uk_addr.address_uuid
	AND t_sub.report_date = uk_addr.report_date
LEFT JOIN db1.entries_installation_points_dir_partner_ch AS enr_d_p 
    ON enr_d_p.address_uuid = uk_addr.address_uuid 
WHERE uk_addr.report_date in d_range
ORDER BY report_date DESC
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [34]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_uk_adresses_subscribtions
    ORDER BY report_date
    limit 10
    """

ch.query_run(query_text)


report_date,partner_uuid_uk,name,address_uuid,partner_uuid,partner_uk_email,company_name,partner_lk,activated_citizen_id,citizen_id_in_flat_with_subscriptions,subscribed_citizen_id,payments_amount,city,full_address,flats_count,entrance_number,building_number,streat_name
date,str,str,str,str,str,str,i64,i64,i64,i64,i64,str,str,i64,str,str,str
2025-08-20,"""0076e1b0-72c3-4783-9306-93d299…","""Жилкомфорт 105 кв""","""cb0afec6-4041-4c9b-b9fd-9cd62a…","""2ac02900-f143-4412-8f8c-df41ce…","""zhilcom03sputnik@mail.ru""","""Дом плюс""",123231,25,2,1,0,"""Улан-Удэ""","""Россия, Республика Бурятия, го…",20,"""п. 1""","""д. 31""","""микрорайон 105-й"""
2025-08-20,"""0076e1b0-72c3-4783-9306-93d299…","""Жилкомфорт 105 кв""","""acc6f2b2-98dc-4657-8f07-0c9b7a…","""2ac02900-f143-4412-8f8c-df41ce…","""zhilcom03sputnik@mail.ru""","""Дом плюс""",123231,43,3,1,0,"""Улан-Удэ""","""Россия, Республика Бурятия, го…",20,"""п. 2""","""д. 31""","""микрорайон 105-й"""
2025-08-20,"""008d558d-fa9b-499f-b0a4-b4a00b…","""УК ""НЭК ЭНЕРГО""""","""79be1203-eab6-4366-9ccc-406080…","""fd20414d-c1b4-4d16-a92f-d3b05d…","""nek-energe@yandex.ru""","""ООО ""Технет""""",156864,0,0,0,0,"""""","""""",0,"""п. 1""","""д. 113А""","""улица Ленинградская"""
2025-08-20,"""008d558d-fa9b-499f-b0a4-b4a00b…","""УК ""НЭК ЭНЕРГО""""","""b553d9d5-6405-40da-a11a-291e30…","""fd20414d-c1b4-4d16-a92f-d3b05d…","""nek-energe@yandex.ru""","""ООО ""Технет""""",156864,0,0,0,0,"""""","""""",0,"""п. 2""","""д. 113А""","""улица Ленинградская"""
2025-08-20,"""008d558d-fa9b-499f-b0a4-b4a00b…","""УК ""НЭК ЭНЕРГО""""","""7a146051-a10b-4a6b-9b1c-1c7eb7…","""fd20414d-c1b4-4d16-a92f-d3b05d…","""nek-energe@yandex.ru""","""ООО ""Технет""""",156864,16,9,5,0,"""Вологда""","""Россия, Вологодская область, г…",11,"""п. 4""","""д. 72""","""улица Гагарина"""
2025-08-20,"""008d558d-fa9b-499f-b0a4-b4a00b…","""УК ""НЭК ЭНЕРГО""""","""488347f8-5cb9-4656-82a4-4049c2…","""fd20414d-c1b4-4d16-a92f-d3b05d…","""nek-energe@yandex.ru""","""ООО ""Технет""""",156864,46,17,11,0,"""Вологда""","""Россия, Вологодская область, г…",30,"""п. 5""","""д. 72""","""улица Гагарина"""
2025-08-20,"""008d558d-fa9b-499f-b0a4-b4a00b…","""УК ""НЭК ЭНЕРГО""""","""e1196f35-b652-4338-a4d9-a4b7a1…","""fd20414d-c1b4-4d16-a92f-d3b05d…","""nek-energe@yandex.ru""","""ООО ""Технет""""",156864,24,6,4,0,"""Вологда""","""Россия, Вологодская область, г…",19,"""п. 6""","""д. 72""","""улица Гагарина"""
2025-08-20,"""00b3faa7-ed4f-4ee5-93a1-7f0461…","""Физкультурная 9/3""","""d903e192-73b0-4e7a-85d9-39775a…","""af8b5bf4-0108-4a0c-acc2-096f94…","""gulaimnika@gmail.com""","""ТОО РИТМ ПРО""",120074,60,0,0,0,"""Семей""","""Казахстан, область Абай, город…",30,"""п. 1""","""д. 9/3""","""Физкультурная"""
2025-08-20,"""00b3faa7-ed4f-4ee5-93a1-7f0461…","""Физкультурная 9/3""","""36c91545-1b43-4e5a-8ed4-91bc2e…","""af8b5bf4-0108-4a0c-acc2-096f94…","""gulaimnika@gmail.com""","""ТОО РИТМ ПРО""",120074,61,0,0,0,"""Семей""","""Казахстан, область Абай, город…",30,"""п. 2""","""д. 9/3""","""Физкультурная"""


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_uk_adresses_subscribtions DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [30]:
query_text = """--sql
    DROP TABLE db1.t_uk_adresses_subscribtions_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [31]:
query_text = """--sql
    DROP TABLE db1.t_uk_adresses_subscribtions
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_uk_adresses_subscribtions_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
897986,"""58d516b8-81ce-47ef-ab2c-c0196a…"
